In [70]:
# Definir colores usando códigos ANSI
orange = "\033[38;5;208m"
red = "\033[91m"
blue = "\033[34m"
green = "\033[32m"
yellow = "\033[33m"
gray = "\033[90m"
reset = "\033[0m"  # <-- to reset color


# Función helper para imprimir con color
def pprint(color="", content="", bg_color="", end="\n"):
    print(f"{color}{content}{reset}", end=end)


# Ejemplo de uso
pprint(orange, "Hi mom!")
pprint(red, "Hi mom!")
pprint(blue, "Hi mom!")
pprint(green, "Hi mom!")
pprint(yellow, "Hi mom!")
pprint(gray, "Hi mom!")

Hi mom!
Hi mom!
Hi mom!
Hi mom!
Hi mom!
Hi mom!


In [1]:
%pip install -qU rapidocr-onnxruntime langchain langchain_community langchain_core pymupdf

Note: you may need to restart the kernel to use updated packages.


In [13]:
import os
from langchain.schema import Document
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders.parsers import RapidOCRBlobParser
from tqdm import tqdm
from typing import List

In [14]:
PDF_DIR = "../../../notificaciones"

In [15]:
def directory_loader(directory_path: str) -> List[Document]:
    """LOADS PDF DOCUMENTS FROM A GIVEN DIRECTORY WITH PROGRESS INDICATOR."""

    if not os.path.exists(directory_path):
        raise ValueError(
            f"directory_loader() >>> DIRECTORY {directory_path} DOESN'T EXIST."
        )

    loaded_docs: List[Document] = []

    # SEARCH IN THE GIVEN DIRECTORY FOR EACH PDF FILE IN IT AND GETS ITS PATH
    pdf_files_info = []
    for parent_dir_path, _, files in os.walk(directory_path):
        for filename in files:
            if filename.endswith(".pdf"):
                file_path = os.path.join(parent_dir_path, filename)
                pdf_files_info.append(file_path)

    # LOADS EACH PDF FILE: FILE --> LIST[DOCUMENT]
    for file_path in tqdm(pdf_files_info, desc="LOADING FILES"):
        loaded_file = PyMuPDFLoader(
            file_path,
            mode="page",
            images_inner_format="text",
            images_parser=RapidOCRBlobParser(),
        ).load()

        # DELETES UNNECESSARY METADATA AND APPENDS THE LOADED FILE TO THE LIST OF LOADED DOCS
        for page in loaded_file:
            """
            Other metadata's fields:
                - file_path
                - format
                - producer
                - creator
                - creationdate
                - author
                - subject
                - keywords
                - moddate
                - trapped
                - modDate
                - creationDate
                - page
                - total_pages
            """

            page.metadata = {
                "source": page.metadata["source"],
                "filename": page.metadata["title"],
            }

        loaded_docs.append(loaded_file)

    return loaded_docs

In [37]:
docs = directory_loader(PDF_DIR)

LOADING FILES: 100%|██████████| 6/6 [00:02<00:00,  2.58it/s]


In [17]:
docs[0]

[Document(metadata={'source': '../../../notificaciones/RES 11-06-2024 -  SE CONCEDE RECURSO EN RELACION.pdf', 'filename': 'Despacho CIV 1950/2024 - SE CONCEDE RECURSO EN RELACION'}, page_content='#38617791#415584608#20240611104415348\nPoder Judicial de la Nación\nJUZGADO CIVIL 58\n1950/2024\nPROVINCIA ART SA c/ INTEGRITY SEGUROS ARGENTINA \nS.A s/DILIGENCIAS PRELIMINARES\nBuenos Aires,     de  \n.- \njunio de 2024\nIGM\nConcédese en relación el recurso de  apelación interpuesto\ncontra el  pronunciamiento de fs. 16.\nOportunamente, de estar en condiciones,  elévense los autos al\nTribunal de Alzada, en la forma  de estilo.\n \nDigitally signed by MARIA DI\nFILIPPO\nDate: 2024.06.11 13:06:18 ART\nSignature Not Verified')]

In [72]:
for index, doc in enumerate(docs):
    for i, page in enumerate(doc):
        pprint(color=orange, content=f"> DOC N°: {index}")
        pprint(color=blue, content=f"> PAGE N°: {i}", end="\n\n")
        pprint(color=green, content=f"> METADATA:")
        pprint(content=page.metadata["filename"], end="\n\n")
        pprint(color=yellow, content=f"> CONTENT:")
        pprint(content=page.page_content, end="\n\n")
        pprint(color=gray, content=f"{'-'*20}\n")

> DOC N°: 0
> PAGE N°: 0

> METADATA:
Despacho CIV 1950/2024 - SE CONCEDE RECURSO EN RELACION

> CONTENT:
#38617791#415584608#20240611104415348
Poder Judicial de la Nación
JUZGADO CIVIL 58
1950/2024
PROVINCIA ART SA c/ INTEGRITY SEGUROS ARGENTINA 
S.A s/DILIGENCIAS PRELIMINARES
Buenos Aires,     de  
.- 
junio de 2024
IGM
Concédese en relación el recurso de  apelación interpuesto
contra el  pronunciamiento de fs. 16.
Oportunamente, de estar en condiciones,  elévense los autos al
Tribunal de Alzada, en la forma  de estilo.
 
Digitally signed by MARIA DI
FILIPPO
Date: 2024.06.11 13:06:18 ART
Signature Not Verified

--------------------

> DOC N°: 1
> PAGE N°: 0

> METADATA:
Despacho CIV 1950/2024 - ESCRITOS. DESPACHO DE ESCRITO SUELTO

> CONTENT:
#38617791#411330538#20240510102530103
Poder Judicial de la Nación
JUZGADO CIVIL 58
1950/2024
PROVINCIA ART SA c/ INTEGRITY SEGUROS ARGENTINA 
S.A s/DILIGENCIAS PRELIMINARES
Buenos Aires,     de  
.- 
mayo de 2024
IGM
Téngase presente para provee